## Importing libs & exploration

In [72]:
import json
import os
import random
import numpy as np
import pandas as pd
import gensim
import nltk

from gensim.models import CoherenceModel
from multiprocess import Pool
from tqdm import tqdm 
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
from string import punctuation

In [73]:
SOCIAL_MEDIA = 'Medusa'

with open(f'data_31.03/{SOCIAL_MEDIA}/result.json') as json_data:
    data = json.load(json_data)

In [74]:
all_types = np.unique([t['type'] for m in data['messages'] for t in m['text'] if type(t) is dict])
all_types

array(['bold', 'email', 'hashtag', 'italic', 'link', 'mention', 'phone',
       'text_link', 'underline'], dtype='<U9')

In [75]:
data['messages'][100:105]

[{'id': 51766,
  'type': 'message',
  'date': '2022-02-24T11:11:30',
  'from': 'Медуза — LIVE',
  'from_id': 'channel1036240821',
  'text': [{'type': 'bold', 'text': 'Главное на 12:00 мск\n'},
   '\nДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА\n\n➤ Минобороны Украины заявило о шестом сбитом российском военном самолете. Российская сторона потери отрицает\n\n➤ Власти Украины сообщили первые данные о жертвах среди мирного населения: в результате обстрелов в разных регионах страны погибли девять человек\n\n➤ Нацбанк Украины ограничил снятие наличных и остановил работу валютного рынка. Введен мораторий на трансграничные платежи и на снятие средств резидентами РФ\n\n➤ Обвал на Московской бирже продолжается. Индекс РТС потерял почти 50%, индекс МосБиржи обрушился на 45%\n\n➤ Литва ввела ЧС из-за военных действий на терр

In [76]:
all_email = np.unique([t['text'] for m in data['messages'] for t in m['text'] if type(t) is dict and t['type'] == 'email'])
all_email

array(['photodoc@meduza.io', 'rusnorm@thebell.io'], dtype='<U18')

'ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА'## To csv
Strange invisible endline symbol

In [143]:
from demoji import replace 

def clean_str(input_str):
    return replace(input_str.strip().replace("\n", ""), '')


BAD_TEXT_TYPES = ['email', 'mention', 'strikethrough', 'hashtag', 'code', 'link', 'text_link']
UKR_CHARS = ['ї', 'є', 'і', 'ʼ', 'ґ']
FOREIGN_AGENT_MESSAGE = 'ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РАСПРОСТРАНЕНО ИНОСТРАННЫМ СРЕДСТВОМ МАССОВОЙ ИНФОРМАЦИИ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА, И (ИЛИ) РОССИЙСКИМ ЮРИДИЧЕСКИМ ЛИЦОМ, ВЫПОЛНЯЮЩИМ ФУНКЦИИ ИНОСТРАННОГО АГЕНТА'

extracted_data = {
    'date': [],
    'text': []
}


ukr_messages_inds = []

for i, m in tqdm(enumerate(data['messages']), total=len(data['messages'])):
    texts = m['text']
    text = ''
    
    if isinstance(texts, str):
        text += clean_str(texts)
    else:
        for t in texts:
            sub_text = ''

            if type(t) is dict and t['type'] not in BAD_TEXT_TYPES and len(t['text']) > 0:
                sub_text = t['text']

            if type(t) is str and len(t) > 0:
                sub_text = t

            sub_text = clean_str(sub_text).strip()

            if len(sub_text) == 0:
                continue
                
            if sub_text[-1] not in ['!', '?', '.', ',', ':', ';']:
                sub_text += '. '
            
            text += sub_text
            
        text = text.replace('\xa0', ' ').replace(FOREIGN_AGENT_MESSAGE, '')
        
    if text != '' and len(text) > 25:
        extracted_data['date'].append(m['date'])
        extracted_data['text'].append(text)
    
    if any(c in UKR_CHARS for c in text):
        ukr_messages_inds.append(len(extracted_data['text']) - 1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4460/4460 [00:13<00:00, 335.71it/s]


In [173]:
np.array(extracted_data['text'])[ukr_messages_inds].shape

IndexError: index 4231 is out of bounds for axis 0 with size 4219

In [145]:
extracted_data['date'] = np.delete(extracted_data['date'], ukr_messages_inds)
extracted_data['text'] = np.delete(extracted_data['text'], ukr_messages_inds)

In [146]:
df = pd.DataFrame(extracted_data)
df['date'] = pd.to_datetime(df['date'])
df.to_csv(f'./data_31.03/{SOCIAL_MEDIA}/dataset.csv', index=False)